# Obtenir dataset pel·lícules

En aquest codi anem a obtenir un dataset de pel·lícules.
Realitzarem els següents passos:
- Carregar llibreries.
- Realitzar web scraping sobre la web https://www.themoviedb.org/movie/
- Capturar el llistat de totes les pel·lícules i obtenir els enllaços
- Accedir a cada pàgina de la pel·lícula i agafar les dades
- Tractar aquestes dades i guardar-les en un dataset.
- Visualitzar el dataset per veure si s'ha creat correctament.

### Carregar llibreries

In [2]:
from selenium.webdriver.chrome.options import Options 
from selenium import webdriver

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd

### Iniciar consulta a la web

In [74]:
url_base = "https://www.themoviedb.org"

options = Options()
# Afegirem l'opció --headless per a que s'execute en segon planol i no obriga el navegador
# options.add_argument("--headless")

# Buscarem les pelicules en la versio en espanyol de la web
options.add_argument("--lang=es-ES");

driver = webdriver.Chrome("chromedriver.exe", options=options)
driver.get(url_base+"/movie")

### Aplicar filtres

Si volem, podem aplicar certs filtres per a l'obtenció d'un tipus de pel·lícules en concret

In [75]:
# Obrir les pestanyes a l'esquerra per tal de fer la selecció dels filtres
for item in driver.find_elements_by_xpath(".//*[@class='filter_panel card closed']"):
    ActionChains(driver).move_to_element(item).click().perform()
time.sleep(1)

In [154]:
# Filtrar per gènere - Fem l'exemple amb el gènere 'Acción'
accio = driver.find_element_by_link_text("Acción")
driver.execute_script("arguments[0].click();", accio)

In [76]:
# Filtrar per plataforma de streaming - Fem l'exemple amb Netflix
Netflix = driver.find_element_by_xpath("//a[@title='Netflix']")
driver.execute_script("arguments[0].click();", Netflix)

Creem una funció per seleccionar una dade en el calendari popup

In [77]:
# Funció per obtindre el mes en numero
def get_mes_numero(mes):
    mesos = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]
    return mesos.index(mes) + 1

# Funció per buscar una dada en el calendari kendoDatePicker de la web
def clic_data_calendari(driver_selenium, id_container, dia_objectiu, mes_objectiu, any_objectiu, posicio_calendari):
    
    # Fer clic per obrir el calendari
    dada_input = driver.find_element_by_id(id_container)
    next_sibling = driver.execute_script("return arguments[0].nextElementSibling", dada_input)
    driver.execute_script("arguments[0].click();", next_sibling)
    time.sleep(1)

    # Obtindre els elements de següent i anterior mes
    next_link = driver.find_elements_by_class_name("k-next-view")[posicio_calendari]
    prev_link = driver.find_elements_by_class_name("k-prev-view")[posicio_calendari]

    # Obtenim 'mes any' que será el indicador que estem en la vista adecuada del calendari
    mes_any_objectiu = mes_objectiu + ' ' + any_objectiu
    mes_objectiu_num = get_mes_numero(mes_objectiu)
    
    # Calculem les variables actuals del calendari de la web
    mes_any_calendari = driver.find_elements_by_class_name("k-title")[posicio_calendari].get_attribute("innerHTML")
    mes_calendari = mes_any_calendari.split()[0]
    mes_calendari_num = get_mes_numero(mes_calendari)
    any_calendari = mes_any_calendari.split()[1]
    
    # Anem desplazant el calendari fins trobar el mes corresponent
    while(mes_any_calendari != mes_any_objectiu):
        if(any_calendari == any_objectiu):
            if(mes_calendari_num > mes_objectiu_num):
                driver.execute_script("arguments[0].click();", prev_link)
                time.sleep(0.5)
            else:
                driver.execute_script("arguments[0].click();", next_link)
                time.sleep(0.5)
        elif(int(any_calendari) > int(any_objectiu)):
            clics_mesos = (int(any_calendari) - int(any_objectiu))*12
            for i in range(clics_mesos):
                driver.execute_script("arguments[0].click();", prev_link)
                time.sleep(0.5)
        else:
            clics_mesos = (int(any_objectiu) - int(any_calendari))*12
            for i in range(clics_mesos):
                driver.execute_script("arguments[0].click();", next_link)
                time.sleep(0.5)
        
        # Actualitzem els valors del calendari        
        mes_any_calendari = driver.find_elements_by_class_name("k-title")[posicio_calendari].get_attribute("innerHTML")
        mes_calendari = mes_any_calendari.split()[0]
        mes_calendari_num = get_mes_numero(mes_calendari)
        any_calendari = mes_any_calendari.split()[1]
      
    if(mes_any_calendari == mes_any_objectiu):
        dia_container = driver.find_element_by_link_text(dia_objectiu) 
        driver.execute_script("arguments[0].click();", dia_container)

In [78]:
# Clic en dada 'desde'
clic_data_calendari(driver, 'release_date_gte', '8', 'febrero', '2019', 0)
time.sleep(1)

# Clic en dada 'hasta'
clic_data_calendari(driver, 'release_date_lte', '19', 'agosto', '2020', 1)
time.sleep(1)

In [79]:
# Realitzar búsqueda
time.sleep(1)
buscar = driver.find_element_by_link_text("Buscar")
driver.execute_script("arguments[0].click();", buscar)
time.sleep(1)

### Obtenir les pel·lícules del llistat.

Utilitzarem Selenium per a interactuar amb la web

Primer farem clic en "Mostrar más" i després anirem carregant les dades mitjançant l'scroll.

Definim un maxim de vegades que farem scroll

In [80]:
# Fer click en "Mostrar más"
element = driver.find_element_by_link_text("Mostrar más") 
driver.execute_script("arguments[0].click();", element)
time.sleep(1)

# Fer scroll per capturar dades conforme es van carregant
url_single_movies = []
max_iterations = 0
iter=1
while max_iterations < 1:
    scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
    # A cada iteració anem augmentant l'scroll
    height=4200*iter
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    # Si hem arribat al final de la pàgina, parem el bucle
    if height > scroll_height:
        print('End of page')
        break
    time.sleep(random.randint(1, 3))
    max_iterations+=1
    iter+=1

body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML') 

# Agafem les pelicules
soup = BeautifulSoup(source, "html.parser")
movies = soup.find_all("div", {"class": "card style_1"})

print(len(movies))

60


### Obtenir informació individual de cada pel·lícula

Primer recorrerem l'HTML obtingut i capturarem els enllaços a la fitxa de cada pel·lícula

In [81]:
# Capturar els enllaços
for movie in movies:
    title = movie.h2.a["href"] 
    url_single_movies.append(title)

Creem algunes funcions auxiliars que ens ajudaran a tractar les dades de la fitxa. A més, si en un futur l'estructura de la pàgina que estem fent web scraping canvia, podem modificar més fàcilment la part que correspongui.

Si no existeix algun valor retornarem "?" en tots els camps excepte amb "Puntuación" que retornarem un -1.

In [82]:
# Get Título
def get_titulo(movie):
    try:
        return movie.find("h2").a.contents[0]
    except AttributeError:
        return "?"

# Get Título original
def get_titulo_otiginal(movie):
    try:
        titulo_container = movie.find("p", {"class": "wrap"}).get_text()
        return titulo_container.split("Título original", 1)[1]
    except AttributeError:
        return "?"

# Get Idioma original
def get_idioma_original(movie):
    try:
        idioma_container = movie.get_text().split("Idioma original", 1)
        return idioma_container[1].split("\n", 1)[0]
    except AttributeError:
        return "?"

# Get Duración
def get_duracion(movie):
    try:
        return movie.find("span", {"class":"runtime"}).text.strip()
    except AttributeError:
        return "?"

# Get Géneros
def get_generos(movie):
    try:
        generos_container = movie.find("span", {"class":"genres"}).find_all("a")
        generos_container = [generos_container[genre].text for genre in range(len(generos_container))]
        return ", ".join(generos_container)
    except AttributeError:
        return "?"
    
# Get Directores
def get_directores(movie):
    try:
        directores = []
        people_container = movie.select("ol.people.no_image li")
        for people in people_container:
            cargo = people.find("p", {"class":"character"}).text
            if "Director" in cargo:
                directores.append(people.find("p").text)
        return ", ".join(directores)
    except AttributeError:
        return "?"

# Get Sinopsis
def get_sinopsis(movie):
    try:
        return movie.find_all("div",{"class": "overview"})[0].text.strip()
    except AttributeError:
        return "?"

# Get Fecha de estreno
def get_fecha_estreno(movie):
    try:
        return movie.find("span", {"class": "release"}).get_text().strip().split(" ", 1)[0]
    except AttributeError:
        return "?"

# Get Clasificación por edad
def get_clasificacion_edad(movie):
    try:
        return movie.find("span", {"class": "certification"}).get_text().strip()
    except AttributeError:
        return "?"

# Get Imagen cartel
def get_img_cartel(movie):
    try:
        return movie.find("img").attrs['src']
    except AttributeError:
        return "?"

# Get Puntiación
def get_puntuacion(movie):
    try:
        return list(movie.find("span",{"class":"icon"}).attrs.values())[0][1][-2:]
    except AttributeError:
        return -1
    
# Get Streaming
def get_streaming(movie):
    try:
        streaming = movie.find("div", {"class":"provider"}).find("img").attrs['alt']
        streaming = streaming.split(" en ")
        return streaming[-1]
    except AttributeError:
        return "?"

Obrim el fitxer csv on guardarem les dades obtingudes i escrivim la primera línea que sera el nom de les columnes

In [83]:
data_file = open('dataset_peliculas.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(data_file, delimiter=";") 
csv_writer.writerow(["Título","Título original","Idioma original","Duración","Géneros","Directores","Sinopsis","Fecha de estreno","Clasificación por edad","Imagen cartel","Puntuación", "Streaming"]) 

152

Accedim a la fitxa de cada pel·lícula i anem capturant les dades i guardant-les en el fitxer csv

In [84]:
# Recorrem els enllaços obtinguts i anem obtinguent informació de cada pel·lícula
for url_movie in url_single_movies[0:5]:
    driver.get(url_base+url_movie)
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML') 
    soup = BeautifulSoup(source, "html.parser")

    # Contenidors de la informació que volem capturar
    poster_wrapper = soup.find("div", {"class": "poster_wrapper"})
    header_wrapper = soup.find("div", {"class": "header_poster_wrapper"})
    column_wrapper = soup.find("section", {"class": "left_column"})

    movie =  [
        get_titulo(header_wrapper), # Título
        get_titulo_otiginal(column_wrapper), # Título original
        get_idioma_original(column_wrapper), # Idioma original
        get_duracion(header_wrapper), # Duración
        get_generos(header_wrapper), # Géneros        
        get_directores(header_wrapper), # Directores
        get_sinopsis(header_wrapper), # Sinopsis
        get_fecha_estreno(header_wrapper), # Fecha de estreno
        get_clasificacion_edad(header_wrapper), # Clasificación por edad
        url_base+get_img_cartel(poster_wrapper), # Imagen cartel
        get_puntuacion(header_wrapper), # Puntuación 
        get_streaming(poster_wrapper) # Streaming
    ]

    csv_writer.writerow(movie)
    
    # Deixem entre 1 i 3 segons de pausa entre cada iteració per no saturar la web
    time.sleep(random.randint(1, 3))

Tanquem el fitxer csv

In [85]:
data_file.close()

Tanquem el driver de selenium

In [73]:
driver.close()

### Comprovació final

Per comprovar que s'han guardat correctament, carreguem el csv en una variable i vegem els primers registres

In [86]:
df_movies = pd.read_csv("dataset_peliculas.csv", sep=";", encoding="utf-8")
df_movies.head(10)

,Título,Título original,Idioma original,Duración,Géneros,Directores,Sinopsis,Fecha de estreno,Clasificación por edad,Imagen cartel,Puntuación,Streaming
0,Tyler Rake,Extraction,Inglés,1h 56m,"Drama, Acción, Suspense",Sam Hargrave,"Tyler Rake, un intrépido mercenario que ofrece...",24/4/2020,18,https://www.themoviedb.org/t/p/w300_and_h450_b...,74,Netflix
1,Bob Esponja: Un héroe al rescate,The SpongeBob Movie: Sponge on the Run,Inglés,1h 35m,"Animación, Fantasía, Aventura, Comedia, Familia",Tim Hill,"Cuando desaparece su amigo Gary, Bob Esponja s...",14/8/2020,APTA,https://www.themoviedb.org/t/p/w300_and_h450_b...,78,Netflix
2,Código 8,Code 8,Inglés,1h 39m,"Ciencia ficción, Acción, Crimen, Suspense, Drama",Jeff Chan,En Lincoln City algunos habitantes tienen habi...,2/7/2020,NR,https://www.themoviedb.org/t/p/w300_and_h450_b...,62,Netflix
3,6 en la sombra,6 Underground,Inglés,2h 8m,"Acción, Suspense",Michael Bay,¿Cuál es la mejor parte de estar muerto? No se...,13/12/2019,16,https://www.themoviedb.org/t/p/w300_and_h450_b...,63,Netflix
4,La tierra y la sangre,La Terre et le Sang,Francés,1h 20m,"Acción, Suspense",Julien Leclercq,Tras años dirigiendo un próspero aserradero en...,17/4/2020,NC16,https://www.themoviedb.org/t/p/w300_and_h450_b...,63,Netflix
